In [1]:
using Microsoft.Azure.Cosmos;

var cosmosClient = new CosmosClient(Cosmos.Endpoint, Cosmos.Key);

Database database = await cosmosClient.CreateDatabaseIfNotExistsAsync("RetailIngest");
Container container = await database.CreateContainerIfNotExistsAsync("WebsiteMetrics", "/CartID");

In [2]:
%%upload --databaseName RetailIngest --containerName WebsiteMetrics --url https://cosmosnotebooksdata.blob.core.windows.net/notebookdata/websiteData.json

Documents successfully uploaded to WebsiteMetrics 
Total number of documents imported: 
  Success: 2654 
  Failure: 0 
Total time taken : 00:00:04 hours 
Total RUs consumed : 27309.660000001593

In [3]:
public class Record
{
  public string Action { get; set; }
  public decimal Price { get; set; }
  public string Country { get; set; }
  public string Item { get; set; }
}

In [4]:
using System.Collections.Generic;

var query = new QueryDefinition(
    query: "SELECT c.Action, c.Price, c.Country, c.Item FROM c"
);

FeedIterator<Record> feed = container.GetItemQueryIterator<Record>(
    queryDefinition: query
);

var results = new List<Record>();
while (feed.HasMoreResults)
{
    FeedResponse<Record> response = await feed.ReadNextAsync();
    foreach (Record result in response)
    {
        results.Add(result);
    }
}

In [5]:
var dictionary = new Dictionary<string, decimal>();

foreach(var result in results)
{
    dictionary.TryAdd (result.Item, result.Price); 
}

dictionary

System.Collections.Generic.Dictionary`2[System.String,System.Decimal]

In [6]:
results

System.Collections.Generic.List`1[Submission#2+Record]